# Prediction of Protein Interaction Network

Machine learning classification task for predicting whether a protein pair interacts or not

Protein pairs are generated from a test pathogen against human proteins contained in training set

In [2]:
import os
import joblib

import pandas as pd
import numpy as np

from features import domain_features

In [3]:
# Set up directories
parent_dir = os.path.dirname(os.getcwd())

dir_in = os.path.join(parent_dir, 'data')
dir_in = os.path.join(parent_dir, 'data', 'results')

## Obtain pathogen proteins

Pathogen: *Streptococcus pneumoniae* strain D39

Proteome: UP000001452 (Taxonomy ID 373153)